# Homework 6

In [4]:
!pip install pyperplan


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.5 MB/s 


## Problems

### Pyperplan Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [5]:
from pyperplan.pddl.parser import Parser
from pyperplan import grounding, planner
from collections import namedtuple
import numpy as np
import os
import tempfile
import pdb
import math, random


BLOCKS_DOMAIN = """(define (domain prodigy-bw)
  (:requirements :strips)
  (:predicates (on ?x ?y)
               (on-table ?x)
               (clear ?x)
               (arm-empty)
               (holding ?x)
               )
  (:action pick-up
             :parameters (?ob1)
             :precondition (and (clear ?ob1) (on-table ?ob1) (arm-empty))
             :effect
             (and (not (on-table ?ob1))
                   (not (clear ?ob1))
                   (not (arm-empty))
                   (holding ?ob1)))
  (:action put-down
             :parameters (?ob)
             :precondition (holding ?ob)
             :effect
             (and (not (holding ?ob))
                   (clear ?ob)
                   (arm-empty)
                   (on-table ?ob)))
  (:action stack
             :parameters (?sob ?sunderob)
             :precondition (and (holding ?sob) (clear ?sunderob))
             :effect
             (and (not (holding ?sob))
                   (not (clear ?sunderob))
                   (clear ?sob)
                   (arm-empty)
                   (on ?sob ?sunderob)))
  (:action unstack
             :parameters (?sob ?sunderob)
             :precondition (and (on ?sob ?sunderob) (clear ?sob) (arm-empty))
             :effect
             (and (holding ?sob)
                   (clear ?sunderob)
                   (not (clear ?sob))
                   (not (arm-empty))
                   (not (on ?sob ?sunderob)))))
"""

BLOCKS_PROBLEM_1 = """(define (problem bw-simple)
  (:domain prodigy-bw)
  (:objects A B C)
  (:init (clear a) (arm-empty) (on a b) (on-table b))
  (:goal (and (on-table a) (clear b))))
"""

BLOCKS_PROBLEM_2 = """(define (problem bw-sussman)
  (:domain prodigy-bw)
  (:objects A B C)
  (:init (on-table a) (on-table b) (on c a)
                (clear b) (clear c) (arm-empty))
  (:goal (and (on a b) (on b c))))
"""

BLOCKS_PROBLEM_3 = """(define (problem bw-large-a)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9)
  (:init (arm-empty)
         (on 3 2)
         (on 2 1)
         (on-table 1)
         (on 5 4)
         (on-table 4)
         (on 9 8)
         (on 8 7)
         (on 7 6)
         (on-table 6)
         (clear 3)
         (clear 5)
         (clear 9))
  (:goal (and
          (on 1 5)
          (on-table 5)
          (on 8 9)
          (on 9 4)
          (on-table 4)
          (on 2 3)
          (on 3 7)
          (on 7 6)
          (on-table 6)
          (clear 1)
          (clear 8)
          (clear 2)
          )))
"""

BLOCKS_PROBLEM_4 = """(define (problem bw-large-b)
  (:domain prodigy-bw)
  (:objects 1 2 3 4 5 6 7 8 9 10 11)
  (:init (arm-empty)
         (on 3 2)
         (on 2 1)
         (on-table 1)
         (on 11 10)
         (on 10 5)
         (on 5 4)
         (on-table 4)
         (on 9 8)
         (on 8 7)
         (on 7 6)
         (on-table 6)
         (clear 3)
         (clear 11)
         (clear 9))
  (:goal (and
          (on 1 5)
          (on 5 10)
          (on-table 10)
          (on 8 9)
          (on 9 4)
          (on-table 4)
          (on 2 3)
          (on 3 11)
          (on 11 7)
          (on 7 6)
          (on-table 6)
          (clear 1)
          (clear 8)
          (clear 2)
          )))
"""

def get_task_definition_str(domain_pddl_str, problem_pddl_str):
  """Get Pyperplan task definition from PDDL domain and problem

  This function is a lightweight wrapper around Pyperplan.

  Args:
    domain_pddl_str: A str, the contents of a domain.pddl file.
    problem_pddl_str: A str, the contents of a problem.pddl file.

  Returns:
    task: a structure defining the problem
  """
  # Parsing the PDDDL
  domain_file = tempfile.NamedTemporaryFile(delete=False)
  problem_file = tempfile.NamedTemporaryFile(delete=False)
  with open(domain_file.name, 'w') as f:
    f.write(domain_pddl_str)
  with open(problem_file.name, 'w') as f:
    f.write(problem_pddl_str)
  parser = Parser(domain_file.name, problem_file.name)
  domain = parser.parse_domain()
  problem = parser.parse_problem(domain)
  os.remove(domain_file.name)
  os.remove(problem_file.name)

  # Ground the PDDL
  task = grounding.ground(problem)

  return task

def get_task_definition(domain_pddl, problem_pddl):
  """Get Pyperplan task definition from PDDL domain and problem

  This function is a lightweight wrapper around Pyperplan.

  Args:
    domain_pddl_str: A str, the name of a domain.pddl file.
    problem_pddl_str: A str, the name of a problem.pddl file.

  Returns:
    task: a structure defining the problem
  """
  # Parsing the PDDDL
  parser = Parser(domain_pddl, problem_pddl)
  domain = parser.parse_domain()
  problem = parser.parse_problem(domain)

  # Ground the PDDL
  task = grounding.ground(problem)

  return task

def h_add_test(prob_str, expected_h):
  task = get_task_definition_str(BLOCKS_DOMAIN, prob_str)
  h = h_add(task)
  assert h == expected_h, f"Expected h={expected_h}, but got {h}."
  return h

def h_ff_test(prob_str, expected_h):
  task = get_task_definition_str(BLOCKS_DOMAIN, prob_str)
  h = h_ff(task)
  assert h == expected_h, f"Expected h={expected_h}, but got {h}."

### Construct Reduced Planning Graph (RPG)
Given a [Pyperplan Task](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py) instance and an optional state, return (F_t, A_t), two lists as defined in the RPG pseudo-code from lecture.  Note that _fact names_ in the documentation of `Task` are strings such as '(on a b)' and operator instances are instances of the `Operator` class; the `name` of an operator instance is a string such as '(unstack a c)'.  Is state is None, use `task.initial_state`.

For reference, our solution is **13** lines of code.

In [6]:
def construct_rpg(task, state=None):
  """Constructs a Relaxed Planning Graph (RPG) for a given Pyperplan Task and state.  If state is None, use the initial_state of the task.

  Args:
    task: a Pyperplan Task instance
    state: a set of facts (can be None)

  Returns:
    rpg: A pair (F_sets, A_sets).  Note that F_sets has length one longger than A_sets.  F_sets is a list and the item in the list at F[t] should be a set of fact names.  We expect F_set to be something like "[{'(clear a)', ...}, {'(on-table b)',...}, ...]”.  A_sets is a list and the item in the list at A[t] should be a set of operator instance names.  We expect A_sets to be something like "[{'(unstack a b)', ...}, {'(stack a b)', ... }, ... ]"
  """
  if state == None:
    state = task.initial_state
  f = [state]
  a = []
  t = 0
  while not task.goal_reached(f[t]):
    temp = set(f[t])
    act = set()
    for ac in task.operators:
      if ac.applicable(f[t]):
        act.add(ac)
        temp = temp.union(set(ac.add_effects))
    a.append(act)
    f.append(temp)
    if temp == f[t]:
      break
    t+=1
  return f,a


Tests

In [7]:
def construct_rpg_test1():
  task = get_task_definition_str(BLOCKS_DOMAIN, BLOCKS_PROBLEM_1)
  expected_F_sets = [{'(clear a)', '(on a b)', '(on-table b)', '(arm-empty)'}, {'(on-table b)', '(arm-empty)', '(clear a)', '(on a b)', '(holding a)', '(clear b)'}, {'(arm-empty)', '(on-table a)', '(holding a)', '(holding b)', '(on-table b)', '(on a a)', '(clear a)', '(on a b)', '(clear b)'}]
  F_sets, A_sets = construct_rpg(task)
  assert len(F_sets) == len(expected_F_sets), f"Expected F_sets of length {len(expected_F_sets)}, but got {len(F_sets)}."
  for F_set, expected_F_set in zip(F_sets, expected_F_sets):
      assert F_set == expected_F_set, f"Incorrect F_set: {F_set}"
  expected_A_set_strs = [{'(unstack a b)'}, {'(unstack a b)', '(pick-up b)', '(stack a a)', '(stack a b)', '(put-down a)'}]
  assert len(A_sets) == len(expected_A_set_strs), f"Expected A_sets of length {len(expected_A_set_strs)}, but got {len(A_sets)}."
  for A_set, expected_A_set_str in zip(A_sets, expected_A_set_strs):
      A_set_str = {a.name for a in A_set}
      assert A_set_str == expected_A_set_str, f"Incorrect A_set: {A_set_str}, expected {expected_A_set_str}"
  return F_sets, [{a.name for a in A_set} for a in A_sets]

construct_rpg_test1()
def construct_rpg_test2():
  task = get_task_definition_str(BLOCKS_DOMAIN, BLOCKS_PROBLEM_2)
  expected_F_sets = [{'(arm-empty)', '(clear c)', '(on-table a)', '(on c a)', '(clear b)', '(on-table b)'}, {'(clear c)', '(holding b)', '(on-table a)', '(holding c)', '(clear b)', '(clear a)', '(on-table b)', '(arm-empty)', '(on c a)'}, {'(clear c)', '(holding b)', '(on-table a)', '(holding c)', '(on b a)', '(on c c)', '(clear b)', '(holding a)', '(clear a)', '(on-table b)', '(arm-empty)', '(on b b)', '(on b c)', '(on c b)', '(on c a)', '(on-table c)'}, {'(holding b)', '(on-table a)', '(on c c)', '(clear a)', '(on-table b)', '(on b b)', '(on c a)', '(on a b)', '(clear c)', '(holding c)', '(on b a)', '(on a a)', '(clear b)', '(holding a)', '(on a c)', '(arm-empty)', '(on b c)', '(on c b)', '(on-table c)'}]
  F_sets, A_sets = construct_rpg(task)
  assert len(F_sets) == len(expected_F_sets), f"Expected F_sets of length {len(expected_F_sets)}, but got {len(F_sets)}."
  for F_set, expected_F_set in zip(F_sets, expected_F_sets):
      assert F_set == expected_F_set, f"Incorrect F_set: {F_set}"
  expected_A_set_strs = [{'(unstack c a)', '(pick-up b)'}, {'(unstack c a)', '(pick-up b)', '(stack b a)', '(stack b c)', '(put-down c)', '(stack c a)', '(stack c c)', '(put-down b)', '(stack c b)', '(pick-up a)', '(stack b b)'}, {'(stack b a)', '(stack b c)', '(unstack b b)', '(stack c c)', '(stack c b)', '(unstack c b)', '(unstack b c)', '(stack a b)', '(put-down a)', '(stack a c)', '(pick-up a)', '(stack a a)', '(unstack c a)', '(pick-up b)', '(put-down c)', '(stack c a)', '(put-down b)', '(unstack b a)', '(stack b b)', '(pick-up c)', '(unstack c c)'}]
  assert len(A_sets) == len(expected_A_set_strs), f"Expected A_sets of length {len(expected_A_set_strs)}, but got {len(A_sets)}."
  for A_set, expected_A_set_str in zip(A_sets, expected_A_set_strs):
      assert {a.name for a in A_set} == expected_A_set_str, f"Incorrect A_set: {A_set}"
  return F_sets, [{a.name for a in A_set} for a in A_sets]

construct_rpg_test2()
print('Tests passed.')

Tests passed.


### Implement h_add
Given a [Pyperplan Task](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py) instance and a state (set of facts or None), return h_add(state).

For reference, our solution is **4** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `construct_rpg`. You may not need to use all of them.

In [8]:
def h_add(task, state=None):
  """Computes h_add for a given Pyperplan Task and state.  If state is None, use the initial_state of the task.

  Args:
    task: a pyperplan Task instance
    state: a set of facts (can be None)

  Returns:
    h: an integer
  """
  G = task.goals
  F_sets, A_sets = construct_rpg(task, state)
  level_g = {g: min(t for t in range(len(F_sets)) if g in F_sets[t]) for g in G}
  return sum(level_g.values())

Tests

In [9]:
def h_add_test1(): return h_add_test(BLOCKS_PROBLEM_1, 3)

h_add_test1()
def h_add_test2(): return h_add_test(BLOCKS_PROBLEM_2, 5)

h_add_test2()
def h_add_test3(): return h_add_test(BLOCKS_PROBLEM_3, 21)

h_add_test3()
def h_add_test4(): return h_add_test(BLOCKS_PROBLEM_4, 30)

h_add_test4()
print('Tests passed.')

Tests passed.


### Implement h_ff
Given a [Pyperplan Task](https://github.com/aibasel/pyperplan/blob/master/pyperplan/task.py) and a state (set of facts or None), return h_ff(state).

For reference, our solution is **18** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `construct_rpg`. You may not need to use all of them.

In [10]:
def h_ff(task, state=None):
  """Computes h_ff for a given task and state.  If state is None, use the initial_state of the task.

  Args:
    task: a Pyperplan Task instance
    state: a set of facts (can be None)

  Returns:
    h: an integer
  """
  G = task.goals
  A = task.operators
  F_sets, A_sets = construct_rpg(task, state)
  level_p = {p: min(t for t in range(len(F_sets)) if p in F_sets[t]) for p in F_sets[-1]}
  level_a = {a: min((t for t in range(len(A_sets)) if a in A_sets[t]), default=float('inf')) for a in A}
  M = max(level_p[p] for p in G)
  G_by_level = [{g for g in G if level_p[g] == t} for t in range(M+1)]
  selected = set()
  for t in range(M, 0, -1):
    for g in G_by_level[t]:
      sel_a = next(a for a in A if level_a[a] == t-1 and g in a.add_effects)
      assert sel_a
      selected.add(sel_a)
      for p in sel_a.preconditions:
        p_level = level_p[p]
        assert p_level < t, 'Preconditions for action not true at this level'
        G_by_level[p_level].add(p)
  return len(selected)

Tests

In [11]:
def h_ff_test1(): return h_ff_test(BLOCKS_PROBLEM_1, 2)

h_ff_test1()
def h_ff_test2(): return h_ff_test(BLOCKS_PROBLEM_2, 5)

h_ff_test2()
def h_ff_test3(): return h_ff_test(BLOCKS_PROBLEM_3, 12)

h_ff_test3()
def h_ff_test4(): return h_ff_test(BLOCKS_PROBLEM_4, 16)

h_ff_test4()
print('Tests passed.')

Tests passed.


### RRT-Related Utilties
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [12]:
# Useful geometric data structures and methods.
def euclidean_distance(a, b):
  return math.sqrt((a.x - b.x) ** 2 + (a.y - b.y) ** 2)


RobotConfig = namedtuple('RobotConfig', ['x', 'y'])


class Circle(namedtuple('Circle', ['x', 'y', 'r'])):
  def collide(self, config):
    return euclidean_distance(config, self) < self.r


class RobotPlanningProblem:
  """
  A robot planning problem definition contains the configuration space of the robot
  and all obstacles represented as geometries in the configuration space.

  In this problem set, we will consider a simple 2D map, where the robot configuration
  space is represented as a tuple (x, y).

  In the __init__ definition, we will define the range of x and y values of the robot's configuration,
  as well as a list of obstacles. See the code for details.
  """
  def __init__(self):
    # The ma
    self.x_range = (-10, 10)  # range of the x value.
    self.y_range = (-10, 10)  # range of the y value.
    self.obstacles = [
      Circle(0, 0, 4),
      Circle(-5, -5, 2),
      Circle(3, -4, 3),
      Circle(-3, 4, 1)
    ]

  def sample(self):
    """Generate a random sample in the robot configuration space."""
    return RobotConfig(random.uniform(*self.x_range), random.uniform(*self.y_range))

  def distance(self, a, b):
    """Return the distance between two robot configurations.

    Args:
      a (RobotConfig): the first robot configuration.
      b (RobotConfig): the second robot configuration.

    Return:
      distance (float): the distance between two configurations.
    """
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

  def collide(self, x):
    """Test if the input robot configuration collides with some obstacles.

    Args:
      x (RobotConfig): the input configuration.

    Returns:
      collision (bool): True if a collision happens between the input configuration and one of the obstacle in the environment.
    """
    for obs in self.obstacles:
      if obs.collide(x):
        return True
    return False


class RRTNode:
  """Node of a rapidly-exploring random tree.

  Each node is associated with a robot configuration.
  It also keeps track of
    - the parent in the tree.
    - a list of children nodes in the tree.

  If the node itself is the node for an RRT, node.parent will be set to None.
  If the node is a leaf node in an RRT, node.children will be an empty list.
  """

  # config: RobotConfig # Robot state
  # parent: Optional['RRTNode']  # parent node
  # children: List['RRTNode']  # children

  def __init__(self, config: RobotConfig, parent = None):
    self.config = config
    self.parent = parent
    self.children = list()

  def add_child(self, other):
    """
    Register another node as the child of the current node.

    Args:
      other (RRTNode): the other RRTNode.

    Returns:
      self (RRTNode)
    """
    self.children.append(other)
    return self

  def attach_to(self, parent):
    """Attach the current node to another node. This function will:
      - set the parent of self to parent.
      - add self to parent.children.

    Args:
      parent (RRTNode): the parent node.

    Returns:
      self (RRTNode)
    """
    self.parent = parent
    self.parent.add_child(self)
    return self

  def backtrace(self, config=True):
    """Return the path from root to the current node. For example, if the tree is:

    s0 --> s1 -> s2 -> S3 -> s4
       \
        -> s5 -> s6

    Calling s6.backtrace() will return the list: [s0, s5, s6].

    Args:
      config (bool): if True, return a list of robot configurations.
      Otherwise, return the RRTNode list.

    Returns:
      path (List[RobotConfig]): the path from root to the current node.
    """
    path = list()

    def dfs(x):
      nonlocal path
      if x.parent is not None:
        dfs(x.parent)
      path.append(x.config if config else x)

    try:
      dfs(self)
      return path
    finally:
      del dfs

  def __repr__(self):
      return f'RRTNode(config={self.config}, parent={self.parent})'


class RRT:
  """The RRT tree.

  The constructor of RRT takes two parameters:
    problem (RobotPlanningProblem): an instance of the robot planning problem.
      It will define the distance metric in the configuration space.
    roots (List[RRTNode]): a list of RRTNode instances. They are the "roots" of the RRT.
      Note that it is a list of nodes. So if you want to construct an RRT with a single
      root node, please use RRT(problem, [your_single_node]).

  Note that the RRT class does not explicitly manages all nodes in the tree.
  It only keeps track of the root nodes of the tree.
  To get all nodes in an RRT tree, we need to do a tree travesal from the roots.
  We have provided a helper function traverse_rrt_bfs for you to do that.

  There are two useful helper functions in RRT class.
    - extend: which attaches a new node to an existing node in the tree.
    - nearest: which finds the node in the RRT that is most close to the query node.
  See the function docstrings for details.
  """

  def __init__(self, problem, roots):
    if isinstance(roots, RRTNode):
      roots = [roots]
    else:
      assert isinstance(roots, list)

    self.problem = problem
    self.roots = roots
    self.size = len(roots)

  def extend(self, parent, child_config):
    """Create a new RRTNode from a robot configuration and set its parent.

    Basically, it creates a new RRTNode instance from child_config and attach this
    new node to the `parent` node. We recommend you to use this function
    instead of doing it yourself because it will help you keep track of the size
    of the RRT, which may be useful for your debugging purpose.

    Args:
      parent (RRTNode): the parent node (in the current RRT).
      child_config (RobotConig): the robot configuration of the new node.

    Returns:
      child (RRTNode): the new node.
    """
    child = RRTNode(child_config).attach_to(parent)
    self.size += 1
    return child

  def nearest(self, config):
    """Return the RRTNode in the tree that is closest to the target configuration.
    We strongly suggest you to take a look at the function definition to see
    the example usage of function `traverse_rrt_bfs` and `problem.distance`.

    Args:
      config (RobotConfig): the query robot config.

    Returns:
      best_node (RRTNode): a node in the RRT that is closest to the query configuration
        based on the distance metric in the configuration space.
    """
    best_node, best_value = None, np.inf
    for node in traverse_rrt_bfs(self.roots):
      distance = self.problem.distance(node.config, config)
      if distance < best_value:
        best_value = distance
        best_node = node

    return best_node


def traverse_rrt_bfs(nodes):
  """Run breath-first search to return all RRTNodes that are descendants of
  the input list of RRTNodes.

  Args:
    nodes: a list of RRTNodes as the seed nodes.

  Returns:
    a list of RRTNodes that are descendants of the input lists (including the input list).
  """
  queue = nodes.copy()
  results = list()

  while len(queue) > 0:
    x = queue[0]
    queue = queue[1:]
    results.append(x)
    for y in x.children:
      queue.append(y)

  return results


########## Test-Related Tools for RRT ##########


def check_path_smooth(problem, path, max_diff, eps=1e-5):
  """Check if a path is collision-free and satisfies the max_diff constraint."""
  if path is None or len(path) == 0:
    return False
  for config1, config2 in zip(path[:-1], path[1:]):
    if any(abs(config1[i] - config2[i]) > max_diff + eps for i in range(len(config1))):
      return False
  for config in path:
    if problem.collide(config):
      return False
  return True


def check_path_smooth_and_successful(problem, path, initial_config, goal_config, max_diff, eps=1e-5):
  """Check if a path is collision-free, satisfies the max_diff constraint, and connects initial_config and goal_config."""
  return path is not None and len(path) > 0 and np.allclose(path[0], initial_config, atol=eps) and np.allclose(path[-1], goal_config, atol=eps) and check_path_smooth(problem, path, max_diff, eps=eps)

### Extend Path in a Robot Configuration Space (OPTIONAL)
In this section, you will implement an important utility for doing motion planning in a continuous space: extend_path.
The function will take two arguments, `initial_config` and `goal_config` and make a linear interpolation between these
two configurations in the robot's configuration space.

Its output is a path, that is, a list of robot configurations along the line from `initial_config` to `goal_config`.
When the `initial_config` and `goal_config` cannot be directly connected with a line because of collision,
your function should return a "prefix" of the path.
In other words, you should try to move towards the goal until you reach some obstacles in the environment.

Take a close look at the docstring of the `RobotPlanningProblem` class before you implement this function.


For reference, our solution is **15** lines of code.

In [15]:
def extend_path(problem, initial_config, goal_config, max_diff):
  """Construct a linear path in the configuration space from the initial configuration to the goal configuration.

  Your implementation should have two steps.
  Step 1, construct a path that is a linear interpolation between initial_config and goal_config.
  At each step, the movement along x and along y should be both smaller than or equal to max_diff.

  Step 2: check for collisions.
  You should iterate over the configurations in the path generated in step 1,
  if a configuration triggers a collision with an obstacle,
  your algorithm should return the longest prefix path that does not have any collision.
  We have implemented this test for you. But make sure you read the code and understand how it works.

  Note that, even though we have separated linear interpolation and collision checking as two
  separate steps, you can also choose to do them simultaneously in a single for loop.

  Args:
    problem (RobotPlanningProblem): a RobotPlanningProblem instance, used for collision checking.
    initial_config (RobotConfig): the initial configuration. It should be the first element of your return list.
    goal_config (RobotConfig): the target configuration. It should be the last element of your return list (if succeed).
    max_diff (float): the maximum movement in x or y.

  Returns:
    successful: a bool value indicating whether we have found a path from the initial configuration to the goal configuration.
      If a collision happens, you should return False. See the starter code.
    path: a list of RobotConfig: extending from the initial to the goal configuration.
  """
  path = [initial_config]
  # Step 1: Generate a list of robot configurations that connects initial_config and goal_config.
  # Your code only needs to take care of max_diff. You can ignore collision checking for now.
  # Your code here...
  ndims = len(initial_config)
  diffs = [goal_config[i] - initial_config[i] for i in range(ndims)]
  nr_samples = max([int(math.ceil(abs(diff) / max_diff)) + 1 for diff in diffs])
  nr_samples = max(2, nr_samples)
  linear_interpolation = [diff / (nr_samples - 1.0) for diff in diffs]
  path = [initial_config]
  for s in range(1, nr_samples - 1):
    sample = RobotConfig(*[initial_config[i] + s * linear_interpolation[i] for i in range(ndims)])
    path.append(sample)
  path.append(goal_config)

  for i, config in enumerate(path):
    if problem.collide(config):
      return False, path[:i]
  return True, path

Tests

In [16]:
def extend_path_test1():
  problem = RobotPlanningProblem()
  initial_config = RobotConfig(-8, -8)
  goal_config = RobotConfig(8, 8)

  success, path = extend_path(problem, initial_config, goal_config, 0.1)
  assert not success     # you can't reach the goal with a single linear motion.
  assert len(path) > 10  # you should make some progress.
  assert check_path_smooth(problem, path, 0.1)

extend_path_test1()
def extend_path_test2():
  problem = RobotPlanningProblem()
  initial_config = RobotConfig(-8, -8)
  goal_config = RobotConfig(-7, 6)

  success, path = extend_path(problem, initial_config, goal_config, 0.1)
  assert success
  assert check_path_smooth_and_successful(problem, path, initial_config, goal_config, 0.1)

extend_path_test2()
print('Tests passed.')

Tests passed.


### Extend Path Visualization (OPTIONAL)

We have created a visualization tool for you in colab to visualize the path. You can run the following code blocks
and they will generate the visualized the map as well as the path.


In [18]:
problem = RobotPlanningProblem()
initial_config = RobotConfig(-8, -8)
goal_config = RobotConfig(-7, 6)

# we have set max_diff to 0.5 for better visualization.
success, path = extend_path(problem, initial_config, goal_config, 0.5)
# visualize_path(problem, path)

### RRT (OPTIONAL)
In this section, you will implement the RRT algorithm we talked about in class to find a path from one configuration to the goal configuration in a 2D environment.

We have implemented some utility classes (RRTNode, RRT) for you, as well as their corresponding visualization tools.
You should take a close look at their docstrings before you implement this function.  **To run in Colab, you will need an implementation of `extend_path`, if you did not write your own, use the solution in Catsoop.**

**Hint**: recall that there is a hyperparameter in RRT, which balances how frequently we want to test if the goal configuration can be reached from a node in the current
RRT tree. Our suggested value for this hyperparameter is 0.1.


For reference, our solution is **18** lines of code.

In addition to all of the utilities defined at the top of the colab notebook, the following functions are available in this question environment: `extend_path`. You may not need to use all of them.

In [ ]:
def rrt(problem, initial_config, goal_config, max_diff, nr_iterations=100, goal_sample_prob=0.1):
  """Implement RRT algorithm to compute a path from the initial configuration to the target configuration.
  Your implementation should returns a list of robot configurations, starting from initial_config and terminating at goal_config.

  Similar to the definition in extend_path, at each step, the movement along x and along y should be both smaller than or equal to max_diff.
  You should use the extend_path utility that you just implemented to check whether there is a path from one node to another.

  Args:
    problem (RobotPlanningProblem): an RobotPlanningProblem instance, used for configuration sampling and collision checking.
    initial_config (RobotConfig): the initial configuration. It should be the first element of your return list.
    goal_config (RobotConfig): the target configuration. It should be the last element of your return list.
    max_diff (float): the maximum movement in x or y.
    nr_iterations (int): the number of iterations in RRT.
    goal_sample_prob (float): at each iteration, with prob p, you should try to construct a path from a node to the goal configuration.
      Otherwise, you should sample a new configuration and try to constrcut a path from a node to it.

  Returns:
    path: a list of RobotConfig as the trajectory. Return None if we can't find such a path after the specified number of iterations.
    rrt: an RRT object. The root should be the intial_config. There must also exist another node for goal_config.
  """
  raise NotImplementedError("Implement me!")

Tests

In [ ]:
def rrt_test():
  problem = RobotPlanningProblem()
  initial_config = RobotConfig(-8, -8)
  goal_config = RobotConfig(8, 8)

  path, tree = rrt(problem, initial_config, goal_config, 0.5, nr_iterations=200)
  assert check_path_smooth_and_successful(problem, path, initial_config, goal_config, 0.5)

rrt_test()
print('Tests passed.')

### RRT Visualization (OPTIONAL)

We have created a visualization tool for you in colab to visualize the RRT tree during your search. You can run the following code blocks
and they will generate the visualized the map as well as some lines showing the edges in the RRT.

If you run this cell multiple times, you will see different RRT trees due to different random seeds.
You will find that sometimes the tree has very few nodes -- indicating that your RRT algorithm finds a solution very quickly.
Sometimes it takes many iterations to find the solution. This suggests that the algorithm is very sensitive to random seeds,
which is bad for practical usage. Thus, in practice, we shuold always use improved RRT algorithm variants such as BiRRT (bidirectional RRT).

We recommend you try in your colab notebook to see how the hyperparameter `goal_sample_probability` will affect the search process.
Specifically, try to set that probability to 0.05 or 0.2 and visualize your result.


In [ ]:
problem = RobotPlanningProblem()
initial_config = RobotConfig(-8, -8)
goal_config = RobotConfig(8, 8)

# we have set max_diff to 0.5 for better visualization.
path, tree = rrt(problem, initial_config, goal_config, 0.5)
visualize_rrts(problem, [tree], initial_config, goal_config)